In [2]:
import cv2
import numpy as np

In [15]:
# find contours after color based image segmentation 
def get_contours(frame):
  im = frame.astype(dtype = np.uint8)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  hsv_im = cv2.cvtColor(im, cv2.COLOR_RGB2HSV)

  mask = cv2.inRange(hsv_im, (35, 10, 100), (90, 250, 250))
  result = cv2.bitwise_and(im, im, mask=mask)
  result = cv2.GaussianBlur(result, (3, 3), 0)

  kernel = np.ones((3,3),np.uint8)
  opening = cv2.morphologyEx(result, cv2.MORPH_OPEN, kernel)
  opening = cv2.cvtColor(opening, cv2.COLOR_BGR2GRAY)
  contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_NONE)
  return contours

In [14]:
# classification 
def get_shape(approx, cnt):
  answer = None
  colour = None
  if len(approx) == 3:
    colour = (255, 0, 0)
    answer = 'triangle' 
  elif len(approx) >= 6:
    # find circle among polygons
    m = cv2.moments(cnt)
    x = int((m["m10"] / m["m00"]))
    y = int((m["m01"] / m["m00"]))
    dist = np.array([np.linalg.norm(approx[i] - np.array([x, y])) for i in range(len(approx))])
    r = np.mean(dist)
    t = np.mean(abs(dist - r))
    if t < 7.5:
      colour = (0, 0, 250)
      answer = 'circle'
  elif len(approx) == 4:
    # find rectangle among quadrangles
    a = np.array(approx[0] - approx[1])[0]
    b = np.array(approx[1] - approx[2])[0]
    c = np.array(approx[2] - approx[3])[0]
    d = np.array(approx[3] - approx[0])[0]
  
    if (abs(np.dot(a, b))/(np.linalg.norm(a)*np.linalg.norm(b)) < 0.2 and
        abs(np.dot(b, c))/(np.linalg.norm(b)*np.linalg.norm(c)) < 0.2 and
        abs(np.dot(c, d))/(np.linalg.norm(c)*np.linalg.norm(d)) < 0.2):
      answer = 'rectangle'
      colour = (0, 120, 0)

  return(answer, colour)

In [16]:
def draw_result(cnt, image):
  if cv2.contourArea(cnt) > 5000 and cv2.arcLength(cnt, True) > 300:
      peri = cv2.arcLength(cnt, True)
      approx = cv2.approxPolyDP(cnt, 0.025 * peri, True)
      shape, clr = get_shape(approx, cnt)
      if shape and clr:
        cv2.drawContours(image, cnt, -1, clr, 3)
        M = cv2.moments(cnt)
        cX = int((M['m10'] / M['m00']))
        cY = int((M['m01'] / M['m00']))
        cnt = cnt.astype('float').astype('int')
        cv2.putText(image, shape, (cX - 50, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, clr, 2)
  return image

In [17]:
cap = cv2.VideoCapture('drive/MyDrive/It-jim/task1/task1_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('drive/MyDrive/It-jim/task1/output.mp4',fourcc, 30., (1920, 1080))

while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    print("Can't receive frame (stream end?)")
    break

  contours = get_contours(frame)
  for contour in contours:
    frame = draw_result(contour, frame)

  frame = cv2.resize(frame, (frame.shape[1], frame.shape[0]))
  out.write(frame)
  if cv2.waitKey(1) == ord('q'):
    break

out.release()
cap.release()
cv2.destroyAllWindows()

Can't receive frame (stream end?)
